In [1]:
import logging
# Set up logger
logger = logging.getLogger()
logger.handlers = []
ch = logging.StreamHandler()
formatter = logging.Formatter(
    fmt="%(asctime)s (%(levelname)s): %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
)
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel("INFO")

In [2]:
from gemnet.model.gemnet import GemNet
from gemnet.model.utils import read_json

from ase_calculator import Molecule, MDSimulator
from ase.build import molecule as ase_molecule_db

# for visualization
from ase.io.trajectory import Trajectory
import nglview

# Model settings

In [3]:
model_dir = "./pretrained/GemNet-Q/"
weights_file = f"{model_dir}/weights/ckpt"

# Load the model

In [4]:
model_kwargs = read_json(f"{model_dir}/model_kwargs.json")
model_kwargs["scale_file"] = f"{model_dir}" + model_kwargs["scale_file"]

for key, value in model_kwargs.items():
    print(f"{key}: {value}")
    

num_spherical: 7
num_radial: 6
num_blocks: 4
emb_size_atom: 128
emb_size_edge: 128
emb_size_trip: 64
emb_size_quad: 32
emb_size_rbf: 16
emb_size_cbf: 16
emb_size_sbf: 32
emb_size_bil_trip: 64
emb_size_bil_quad: 32
num_before_skip: 1
num_after_skip: 1
num_concat: 1
num_atom: 2
triplets_only: False
num_targets: 1
direct_forces: False
cutoff: 5.0
int_cutoff: 10.0
envelope_exponent: 5
extensive: True
forces_coupled: False
output_init: HeOrthogonal
activation: swish
scale_file: ./pretrained/GemNet-Q/scaling_factors.json


In [5]:
# # uncomment to run in eager mode
# tf.config.experimental_run_functions_eagerly(True) 

model = GemNet(**model_kwargs)
model.load_weights(weights_file)

# Molecule setup
Load from database or build your own by specifying R and Z

In [6]:
mol = ase_molecule_db('C7NH5')
R   = mol.get_positions()
Z   = mol.get_atomic_numbers()

# MD simulation settings

In [7]:
traj_path = "./md_sim.traj"
logfile = "-"               # “-” refers to standard output.
dynamics = "langevin"       # Name of the MD integrator. Implemented: 'langevin' or 'verlet'.
max_steps = 10              # Maximum number of simulation steps.
time = 0.5                  # Integration time step for Newton's law in femtoseconds.
interval = 2                # Write only every <interval> time step to trajectory file.
temperature = 1500          # The temperature in Kelvin.
langevin_friction = 0.002   # Friction coefficient (only used when dynamics is langevin)

# Setup and run the simulation

In [8]:
cutoff = model_kwargs["cutoff"]
int_cutoff = model_kwargs["int_cutoff"]
triplets_only = model_kwargs["triplets_only"]
molecule = Molecule(
    R, Z, cutoff=cutoff, int_cutoff=int_cutoff, triplets_only=triplets_only
)

In [9]:
simulation = MDSimulator(
    molecule, model, 
    dynamics=dynamics, max_steps=max_steps, time=time, temperature=temperature, langevin_friction=langevin_friction,
    interval=interval, traj_path=traj_path, logfile=logfile
)
simulation.run()

2022-03-15 19:41:11 (INFO): Selected MD integrator: Langevin
2022-03-15 19:41:11 (INFO): Save trajectory to ./md_sim.traj


Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000         -74.6765     -77.4343       2.7578  1641.2
0.0010         -74.6480     -77.2796       2.6316  1566.1
0.0020         -74.6280     -76.9330       2.3051  1371.8
0.0030         -74.6367     -76.5432       1.9065  1134.6
0.0040         -74.6339     -76.2188       1.5849   943.2
0.0050         -74.6246     -75.9957       1.3711   816.0


# Visualize simulated trajectory

In [10]:
traj = Trajectory(traj_path)
nglview.show_asetraj(traj)

NGLWidget(count=6)